In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from time import time
from ultralytics import YOLO
from keras.preprocessing import image
from keras.models import Model
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
#from dgl.nn.pytorch.conv import RelGraphConv
from dgl.nn import GraphConv
#import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import cv2
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

global_resize_image=(512,512)

In [5]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [7]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
#filename = 'Flickr_1k.trainImages.txt'
filename='train_split.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 14575


In [8]:
images = 'VG_100K/'

# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [9]:
print(images)

VG_100K/


In [10]:
train_images_file = 'train_split.txt'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [11]:
# Below file conatains the names of images to be used in test data
test_images_file = 'test_split.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

FileNotFoundError: [Errno 2] No such file or directory: 'test_split.txt'

In [9]:
len(test_img)

2489

In [2]:
def preprocess(image_path):
    
    img = image.load_img(image_path, target_size=(224, 224))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [3]:
model=VGG19()
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
print(model.summary())



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Convert to DataFrame
def Convert_to_DataFrame(results):
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()
    data = {'xmin': [box[0] for box in boxes],
        'ymin': [box[1] for box in boxes],
        'xmax': [box[2] for box in boxes],
        'ymax': [box[3] for box in boxes],
        'confidence': confidences,
        'class': classes,
        'name': [names[int(cls)] for cls in classes]}
    output_df = pd.DataFrame(data)
    #print(output_df)
    return output_df

In [14]:
#Object Detection using YOLOv8
def detect_objects(imagePath):
    image=cv2.imread(imagePath)
    image=cv2.resize(image,global_resize_image)
    model = YOLO('yolov8x.pt')
    results=model(image)
    #results=Convert_to_DataFrame(results)
    return results 

In [ ]:
#results=detect_objects(imagePath)
#print(len(results[0].boxes))
#print(results)

In [ ]:
# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    #im_array = im_bgr.numpy()
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()
    plt.imshow(numpy.array(im_rgb))
    plt.show()

    # Save results to disk
    #r.save(filename=f'results{i}.jpg')

In [1]:
#<==================  Region Proposal ======================>


def roi_propose(boxes):
    # Function to calculate IoU
    def calculate_iou(box1, box2):
        # Calculate intersection rectangle coordinates
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        # Calculate area of intersection rectangle
        intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
        
        # Calculate area of both bounding boxes
        box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
        box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
        
        # Calculate IoU
        iou = intersection_area / float(box1_area + box2_area - intersection_area)
        
        return iou

    # Calculate IoU matrix
    num_boxes = len(boxes)
    iou_matrix = np.zeros((num_boxes, num_boxes))
    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            iou_matrix[i, j] = calculate_iou(boxes[i], boxes[j])
            iou_matrix[j, i] = iou_matrix[i, j]  # IoU is symmetric

    # Define threshold for IoU
    iou_threshold = 0.5

    # Merge bounding boxes based on IoU
    merged = set()
    groups = []
    for i in range(num_boxes):
        if i not in merged:
            group = [i]
            for j in range(i + 1, num_boxes):
                if j not in merged and iou_matrix[i, j] >= iou_threshold:
                    group.append(j)
                    merged.add(j)
            groups.append(group)
    # Merge regions completely inside others
    for i, box1 in enumerate(boxes):
        for j, box2 in enumerate(boxes):
            if i != j and calculate_iou(box1, box2) == 1:
                if i in merged:
                    groups.remove(groups[i])
                elif j in merged:
                    groups.remove(groups[j])
    # Create ROIs based on merged bounding boxes
    rois = {}
    for idx, group in enumerate(groups):
        roi_boxes = [boxes[i] for i in group]
        min_x = min(box[0] for box in roi_boxes)
        min_y = min(box[1] for box in roi_boxes)
        max_x = max(box[2] for box in roi_boxes)
        max_y = max(box[3] for box in roi_boxes)
        rois[idx] = (min_x, min_y, max_x, max_y)

    # Function to check if two ROIs overlap
    def is_inside(roi1, roi2):
        x1_min, y1_min, x1_max, y1_max = roi1
        x2_min, y2_min, x2_max, y2_max = roi2
        return (x1_min >= x2_min and y1_min >= y2_min and
                x1_max <= x2_max and y1_max <= y2_max)

    # Function to merge ROIs if one ROI completely encompasses the other
    def merge_overlapping_rois(rois):
        merged_rois = {}
        for idx, roi in rois.items():
            merge = True
            for merged_idx, merged_roi in merged_rois.items():
                if is_inside(roi, merged_roi):
                    # ROI is completely inside another ROI, don't merge
                    merge = False
                    break
                elif is_inside(merged_roi, roi):
                    # Another ROI is completely inside this ROI, merge
                    merged_rois[merged_idx] = roi
                    merge = False
                    break
            if merge:
                merged_rois[idx] = roi
        return merged_rois
    # Merge overlapping ROIs
    merged_rois = merge_overlapping_rois(rois)

    # Print merged ROIs
    for idx, roi in enumerate(merged_rois):
        print(f"ROI {idx}: {roi}")
    '''    
    # Print proposed ROIs
    for roi_id, roi_coords in rois.items():
        print(f"ROI {roi_id}: {roi_coords}")'''
    return merged_rois    


In [13]:
def extract_nodes(image):
    results=detect_objects(image)
    proposed_roi = roi_propose(results[0].boxes.xyxy.tolist()) #Roi calculation
    output_df=Convert_to_DataFrame(results)
    image_nodes=[]
    for i in range(output_df.shape[0]):
        #print(output_df.iloc[i]['name'])
        data={"object_id":i,
        "start_point":(round(output_df.iloc[i]['xmin']),round(output_df.iloc[i]['ymin'])),
        "ending_point":(round(output_df.iloc[i]['xmax']),round(output_df.iloc[i]['ymax'])),
        "label":output_df.iloc[i]['name']
        }
        image_nodes.append(data)
    return image_nodes    

In [ ]:
#<====================  Visualize and save the roi drawn images ===============>

'''import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to visualize ROIs on the image
def visualize_rois(image_path, rois, save_path):
    # Read the image
    image=cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,global_resize_image)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Draw ROIs on the image
    for roi_id, roi_coords in rois.items():
        min_x, min_y, max_x, max_y = roi_coords
        width = max_x - min_x
        height = max_y - min_y
        rect = patches.Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Add text label
        ax.text(min_x, min_y - 5, f'ROI {roi_id}', color='r')

    # Remove axis ticks
    ax.axis('off')

    # Save the image
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)

    # Show the image
    plt.show()

# Define image path and save path
#image_path = 'your_image.jpg'
save_path = 'image_with_rois.jpg'

# Visualize ROIs on the image and save
visualize_rois(imagePath, roi_propose(results[0].boxes.xyxy.tolist()), save_path)'''


In [74]:
def generate_nodeImages(imagePath,nodes):
    image=cv2.imread(imagePath)
    image=cv2.resize(image,(256,256))
    #image = cv2.rectangle(image, (106,19), (188,133), (255, 0, 0) , 2) 
    #crop=image[19:133,106:188]
    #cv2.imshow('image',crop)
    #cv2.waitKey()
    segment_array=[]
    for i in range(len(nodes)):
        start=nodes[i].get('start_point')
        end=nodes[i].get('ending_point')
        #print(end[1])
        a,b,c,d=start[1],end[1],start[0],end[0]
        crop=image[a:b,c:d]
        segment_array.append(crop)
    segment_array.append(image)    
    return segment_array    

In [36]:
def featuresVGG(segment_array):
    VGG_features=[]
    for image in segment_array:
        image=cv2.resize(image,(224,224))
        image=img_to_array(image)
        image=image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        image=preprocess_input(image)
        feature=model.predict(image)
        VGG_features.append(feature)
    return VGG_features    

In [19]:
image=cv2.imread(r"D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\Flicker8k_Dataset\\109202801_c6381eef15.jpg")

In [20]:
nodes=extract_nodes(image)

Using cache found in C:\Users\Riddhick/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-22 Python-3.9.5 torch-2.2.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 29.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [21]:
len(nodes)

3

In [22]:
segment_array=generate_nodeImages(image,nodes)

In [37]:
VGG=featuresVGG(segment_array)

In [28]:
len(segment_array)

4

In [31]:
VGG[2].shape

(1, 1000)

In [49]:
def graph_features(feature_list):
    
    g = dgl.DGLGraph()
    num_nodes = len(feature_list)
    g.add_nodes(num_nodes)
    features = torch.tensor(feature_list, dtype=torch.float32)
    g.ndata['features'] = features
    num_edges = num_nodes*num_nodes-1 
    g.add_edges(torch.randint(num_nodes, (num_edges,)), torch.randint(num_nodes, (num_edges,)))
    g.edata['edge_weights'] = nn.Parameter(torch.rand(num_edges, requires_grad=True))

# Define a Graph Convolutional Network (GCN) model
    class GCN(nn.Module):
        def __init__(self, in_feats, hidden_feats):
            super(GCN, self).__init__()
            self.conv = GraphConv(in_feats, hidden_feats)

        def forward(self, g, features):
            # Perform graph convolution with learnable edge weights
            h = self.conv(g, features)
            h = F.relu(h)
            return h

# Reshape the features tensor
    features = features.squeeze(1)

    # Instantiate the GCN model
    in_feats = features.shape[1] # Number of input features
    hidden_feats = 256 # Number of hidden units
    model = GCN(in_feats, hidden_feats)

    # Forward pass
    output_gcn = model(g, features)
    output=output_gcn.detach().numpy()
    output=output.transpose()
    #b=output.reshape(1,-1)
    return output

In [50]:
x=graph_features(VGG)

In [51]:
x.shape

(256, 4)

In [57]:
def feature_reduction(features):
    pca = PCA(n_components=1)
    pca.fit(features)
    s=StandardScaler()
    x=pca.transform(features)
    x=np.reshape(x,x.shape[0])
    return x

In [59]:
y=feature_reduction(x)

In [60]:
y.shape

(256,)

In [79]:
def complete_extraction(image_path):
    img= cv2.imread(image_path)
    nodes=extract_nodes(img)
    segment_array=generate_nodeImages(img,nodes)
    feature_vectors=featuresVGG(segment_array)
    print(len(nodes))
    gcn_features=graph_features(feature_vectors)
    feature=feature_reduction(gcn_features)

In [87]:
m=complete_extraction("D:\\Paper\\Mimic Human Level Intelligence in Image Descriptioning\\visual_genome\\images\\VG_100K\\112.jpg")

Using cache found in C:\Users\Riddhick/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-22 Python-3.9.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


10


In [85]:
start = time()
encoding_train = {}
for i in range(10):
    print(train_img[i])
print("Time taken in seconds =", time()-start)

D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1001.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1025.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1060.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1068.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\108.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1081.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1082.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1090.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/images/VG_100K\1122.jpg
D:/Paper/Mimic Human Level Intelligence in Image Descriptioning/visual_genome/image